# Nature in a warming World - Data wrangling

This notebook shows the extraction and cleaning of data used in our project.

## Data Sources

**1. International Union for Conservation of Nature [IUCN RED List API -v3](https://apiv3.iucnredlist.org/api/v3/docs)**: Threatened species around the World data. In order to access the data it is necessary to ask for a token.

**2. [World Bank: Climate change knowdledge portal](https://climateknowledgeportal.worldbank.org)**: Climate change related variables, precipitation and average temperatures monthly and yearly for each country.

**3. [National Centers for Environmental Information](https://www.ncdc.noaa.gov/cag/)**: Temperature Anomalies dataset.

**4. [EONET API v.2.1](https://eonet.sci.gsfc.nasa.gov/docs/v2.1)**: Natural events ocurrences around the World.

**5. [NASA Climate](https://climate.nasa.gov/vital-signs/sea-level/)**: Sea level rising measurements.

# Importing libraries

In [1]:
import pandas as pd
from pandas import ExcelFile
import numpy as np
import re
import math
import requests
import json
import geopip
import scipy.stats as ss
import geopandas as gpd
import getpass

from datetime import datetime

pd.set_option('display.max_columns', None)

First, we introduce a function useful for data cleaning process ahead.

In [2]:
def null_cols(data):
    
    """
    This function takes a dataframe df and shows the columns of df that have NaN values
    and the number of them
    
    """
    
    nulls = data.isna().sum()
    return nulls[nulls > 0] / len(data) * 100

## 1. IUCN Red List data set

Because of copyright reasons, we could not extract all the information related to Bird species from the IUCN API. Then, we will use species datasets obtained via FTP from the IUCN and then complete this data with the API.

In [5]:
def collect(file):
    """
    Returns a dataset with all the 'file' data of the Critical endangered species
    """
    c0 = pd.read_csv("../data/raw_data/CR/"+ file +".csv")
    c1 = pd.read_csv("../data/raw_data/Extinct/" + file +".csv")
    c2 = pd.read_csv("../data/raw_data/rest/" + file +".csv")

    return pd.concat([c0, c1, c2])

We collect the ``assessments``, ``countries`` and ``taxonomy`` files by using the function above:

In [117]:
assessments = collect("assessments")
iucnCountries = collect("countries")
taxons= collect("taxonomy")

Having a look to our datasets:

In [118]:
assessments.head(2)

,assessmentId,internalTaxonId,scientificName,redlistCategory,redlistCriteria,yearPublished,assessmentDate,criteriaVersion,language,rationale,habitat,threats,population,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,497499,132523146,Hubbsina turneri,Critically Endangered,"B1ab(i,ii,iii,iv)+2ab(i,ii,iii,iv)",2019,2018-04-17 00:00:00 UTC,3.1,English,The Highland Splitfin is now only known to be ...,<p>This species lives in quiet waters with cur...,The species has a restricted range and it is t...,"The only remaining population, that of Lago Za...",Decreasing,The Highland Splitfin is a freshwater fish spe...,The Highland Splitfin is not a target species ...,Freshwater (=Inland waters),No conservation actions targeting&#160;<em>Hub...,Neotropical,NaN,False,False,Global
1,500479,11058,Kubaryia pilikia,Critically Endangered,B1ab(iii),2012,2011-08-22 00:00:00 UTC,3.1,English,"<p><em><span lang=""EN-US""></em><span lang=""EN-...",This species of snail is ground-dwelling and h...,"<p><span lang=""EN-US"">This species is threaten...",There is no information available on this spec...,Unknown,"<p><span lang=""EN-US"">The geographic range of ...",This species is not utilized.,Terrestrial,"<span lang=""EN-US""><span lang=""EN-US"">Field wo...",Oceanian,2003,True,False,Global


In [119]:
iucnCountries.head(2)

,assessmentId,internalTaxonId,scientificName,code,name,presence,origin,seasonality,formerlyBred
0,500479,11058,Kubaryia pilikia,PW,Palau,Possibly Extinct,Native,NaN,NaN
1,502298,11256,Obovaria haddletoni,US,United States,Possibly Extinct,Native,NaN,NaN


In [120]:
taxons.head(2)

,internalTaxonId,scientificName,kingdomName,phylumName,orderName,className,familyName,genusName,speciesName,infraType,infraName,infraAuthority,subpopulationName,authority,taxonomicNotes
0,132523146,Hubbsina turneri,ANIMALIA,CHORDATA,CYPRINODONTIFORMES,ACTINOPTERYGII,GOODEIDAE,Hubbsina,turneri,NaN,NaN,NaN,NaN,"(de Buen, 1940)",Fernando de Buén described<em>&#160;Hubbsina t...
1,11058,Kubaryia pilikia,ANIMALIA,MOLLUSCA,LITTORINIMORPHA,GASTROPODA,ASSIMINEIDAE,Kubaryia,pilikia,NaN,NaN,NaN,NaN,"Clench, 1948",NaN


From now on we will work on a copy of ``assessments`` dataset.

In [192]:
df = assessments.copy()

# selecting relevant columns
df = df[['assessmentId', 'internalTaxonId', 'scientificName', 'redlistCategory',
       'yearPublished', 'assessmentDate', 'populationTrend', 'systems',
        'realm','scopes']]

#merging the taxon and assessments data
df = df.merge(taxons[["internalTaxonId", "kingdomName", "className"]], on = "internalTaxonId", how = "left")

In [193]:
df.head(2)

,assessmentId,internalTaxonId,scientificName,redlistCategory,yearPublished,assessmentDate,populationTrend,systems,realm,scopes,kingdomName,className
0,497499,132523146,Hubbsina turneri,Critically Endangered,2019,2018-04-17 00:00:00 UTC,Decreasing,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
1,500479,11058,Kubaryia pilikia,Critically Endangered,2012,2011-08-22 00:00:00 UTC,Unknown,Terrestrial,Oceanian,Global,ANIMALIA,GASTROPODA


In [194]:
df.shape

(29284, 12)

In [13]:
#iucnCountries.drop_duplicates(inplace = True)

In [14]:
#df = iucnCountries.merge(df, on= ["assessmentId", "internalTaxonId", "scientificName"], how = "left")

In [128]:
#df.head() 56592 registers with normal dataset for countries

In [195]:
null_cols(df)

populationTrend    29.254883
systems             0.047808
realm               1.782543
dtype: float64

In [196]:
df.drop(columns = ["populationTrend"], inplace = True)

In [197]:
null_cols(df)

systems    0.047808
realm      1.782543
dtype: float64

In [198]:
df.shape

(29284, 11)

In [199]:
df.head()

,assessmentId,internalTaxonId,scientificName,redlistCategory,yearPublished,assessmentDate,systems,realm,scopes,kingdomName,className
0,497499,132523146,Hubbsina turneri,Critically Endangered,2019,2018-04-17 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
1,500479,11058,Kubaryia pilikia,Critically Endangered,2012,2011-08-22 00:00:00 UTC,Terrestrial,Oceanian,Global,ANIMALIA,GASTROPODA
2,500918,1117,Ameca splendens,Critically Endangered,2019,2018-04-18 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
3,502298,11256,Obovaria haddletoni,Critically Endangered,2012,2012-04-11 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA
4,502943,11479,Lemiox rimosus,Critically Endangered,2012,2012-03-12 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA


In [200]:
colnames = {"internalTaxonId": 'taxonid', 
            "scientificName": 'scientific_name', 
            "kingdomName": 'kingdom', 
            "className": 'class', 
            "yearPublished":'published_year',
            "assessmentDate":'assessment_date', 
            "redlistCategory":'category'}

df.rename(columns = colnames, inplace = True)

In [201]:
df.head()

,assessmentId,taxonid,scientific_name,category,published_year,assessment_date,systems,realm,scopes,kingdom,class
0,497499,132523146,Hubbsina turneri,Critically Endangered,2019,2018-04-17 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
1,500479,11058,Kubaryia pilikia,Critically Endangered,2012,2011-08-22 00:00:00 UTC,Terrestrial,Oceanian,Global,ANIMALIA,GASTROPODA
2,500918,1117,Ameca splendens,Critically Endangered,2019,2018-04-18 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
3,502298,11256,Obovaria haddletoni,Critically Endangered,2012,2012-04-11 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA
4,502943,11479,Lemiox rimosus,Critically Endangered,2012,2012-03-12 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA


## 1.1. IUCN Red List API -v3

In order to use this API is necessary to have a token. To encode the password we use the function getpass. 

In [21]:
token = getpass.getpass() 

 ·······································································


Now we define functions to obtain the data from the API.

In [91]:
#categories = ["DD", "LC", "NT", "VU", "EN", "CR", "EW", "EX", "LRlc", "LRnt", "LRcd"]

#base_category = "https://apiv3.iucnredlist.org/api/v3/species/category/EN"
#base_reg = "https://apiv3.iucnredlist.org/api/v3/region/list"
#base_hist = "https://apiv3.iucnredlist.org/api/v3/species/history/{}/:{}" # {name of species}


def extract_spe(keys, data):
    """
    Takes the keys of a dictionary and return the values as columns of the dataframe
    data
    """
    for key in keys:
        data[key] = data.result.apply(lambda x: x[key])
    return data

def extract_country(keys, data):
    
    """
    Takes the keys of a dictionary and return the values as columns of the dataframe
    data
    """
    for key in keys:
        data[key] = data.results.apply(lambda x: x[key])
    return data

def get_iucn_cat(cat):
    """
    Takes a category "cat" and returns all the endangered species
    whose vulnerability status is "cat".
    """
    
    base_category = "https://apiv3.iucnredlist.org/api/v3/species/category/"
    url = base_category + cat + token
    result = requests.get(url)
    df0 = pd.DataFrame(result.json(), columns = ["category",  "result"])
    
    return df0

def get_iucn_country_list():
    
    """
    Returns the country list with iso3 codes from the IUCN data.
    """
    keys_d = ['isocode']
    url = "https://apiv3.iucnredlist.org/api/v3/country/list"+ token
    result = requests.get(url)
    df0= pd.DataFrame(result.json(), columns = ["results"])
    df0 = extract_country(keys_d, df0)
    df0.drop(columns = "results", inplace = True)
    
    return list(df0["isocode"])

def get_iucn_country(name):
    """
    Returns the description of endangered species whose habitat is the country
    "name".
    """
    
    base = "https://apiv3.iucnredlist.org/api/v3/country/getspecies/"

    url = base +name+token
    result = requests.get(url)
    result = pd.DataFrame(result.json(), columns = ["result"])
    result = result.assign(country = name)
    return result

def get_iucn_species(number):
    
    """
    Returns dataset containing information for a species with id "number"
    """
    
    base = "https://apiv3.iucnredlist.org/api/v3/species/id/"

    url = base +str(number)+token
    res = requests.get(url)
    df0 = pd.DataFrame(res.json())
    
    return df0
     
def get_data(list_cat, cat_function):
    
    """
    Returns a dataset listing all species whose comprehensive group is within 'list_cat'
    by using the function cat_function for extracting the data from API.
    """
    
    list_data = []
    for name in list_cat:
        list_data.append(cat_function(name))
    
    df0 = pd.concat(list_data)
    df0.reset_index(inplace= True, drop = True)
    
    return df0
      

In [95]:
countries = get_data(get_iucn_country_list(), get_iucn_country)

We have a look to the ``countries`` dataset:

In [137]:
countries.head()

,result,country,taxonid,scientific_name,category
0,"{'taxonid': 178546, 'scientific_name': 'Ableph...",UZ,178546,Ablepharus deserti,LC
1,"{'taxonid': 135696, 'scientific_name': 'Abrami...",UZ,135696,Abramis brama,LC
2,"{'taxonid': 22725044, 'scientific_name': 'Acan...",UZ,22725044,Acanthis flammea,LC
3,"{'taxonid': 22695490, 'scientific_name': 'Acci...",UZ,22695490,Accipiter badius,LC
4,"{'taxonid': 22695499, 'scientific_name': 'Acci...",UZ,22695499,Accipiter brevipes,LC


It is necessary to extract the data contained in the first column by using the function ``extract_spe`` and also, consider just threatened species.

In [143]:
dfc= extract_spe(['taxonid', 'scientific_name', 'category'], countries).drop(columns = ["result"])

#Filtering the dataset for the threatened species
dfc = dfc[dfc.category.isin(["VU", "EN", "CR", "EW", "EX"])]

In [144]:
dfc.head()

,country,taxonid,scientific_name,category
9,UZ,219,Acinonyx jubatus,VU
10,UZ,251,Acipenser nudiventris Aral Sea stock,EX
38,UZ,162346,Aldrovanda vesiculosa,EN
51,UZ,164598,Alsophylax laevis,CR
52,UZ,47755582,Alsophylax loricatus,VU


In [150]:
dfc.shape

(61469, 6)

Now we will load the data set containing ISO3 codes for countries and the standard names for them as well.

In [145]:
codes = pd.read_csv("../data/raw_data/country_code.csv")   

In [146]:
codes.head()

,Country,ISO2,ISO3
0,Afghanistan,AF,AFG
1,Albania,AL,ALB
2,Algeria,DZ,DZA
3,American Samoa,AS,ASM
4,Andorra,AD,AND


In [147]:
dfc = dfc.merge(codes, left_on = "country", right_on = "ISO2", how = "left").drop(columns = ["country"])

In [148]:
dfc.head()

,taxonid,scientific_name,category,Country,ISO2,ISO3
0,219,Acinonyx jubatus,VU,Uzbekistan,UZ,UZB
1,251,Acipenser nudiventris Aral Sea stock,EX,Uzbekistan,UZ,UZB
2,162346,Aldrovanda vesiculosa,EN,Uzbekistan,UZ,UZB
3,164598,Alsophylax laevis,CR,Uzbekistan,UZ,UZB
4,47755582,Alsophylax loricatus,VU,Uzbekistan,UZ,UZB


In [165]:
dfc.drop_duplicates(inplace = True)

Finally, we can merge ``dfc`` with ``df`` in a temporary datset:           

In [167]:
df0 = (df.merge(dfc, on = ["taxonid", "scientific_name"], how = "right")
         .drop(columns = "category_x")
         .rename(columns = {"category_y": "category"}))

In [168]:
df0.head()

,assessmentId,taxonid,scientific_name,published_year,assessment_date,systems,realm,scopes,kingdom,class,category,Country,ISO2,ISO3
0,497499.0,132523146,Hubbsina turneri,2019.0,2018-04-17 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII,CR,Mexico,MX,MEX
1,500479.0,11058,Kubaryia pilikia,2012.0,2011-08-22 00:00:00 UTC,Terrestrial,Oceanian,Global,ANIMALIA,GASTROPODA,CR,Palau,PW,PLW
2,500918.0,1117,Ameca splendens,2019.0,2018-04-18 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII,CR,Mexico,MX,MEX
3,502298.0,11256,Obovaria haddletoni,2012.0,2012-04-11 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA,CR,United States of America (the),US,USA
4,502943.0,11479,Lemiox rimosus,2012.0,2012-03-12 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA,CR,United States of America (the),US,USA


While verifying the number of null columns we found that there are no date of assesment for about 10% of the data. However, it is possible to obtain this data by using the IUCN API.  

In [169]:
null_cols(df0)

assessmentId        9.348410
published_year      9.348410
assessment_date     9.348410
systems             9.380949
realm              21.875864
scopes              9.348410
kingdom             9.348410
class               9.348410
Country             0.224518
ISO2                0.224518
ISO3                0.224518
dtype: float64

We obtain the list of the missing assessmentIds, so with this list and using the function ``get_data`` and ``get_iucn_species`` we can pull the missing information and complete the dataset df0:

In [183]:
#List of missing Assessmentids:

missingids = list(df0[df0.assessmentId.isna()].taxonid.value_counts().index)

#calling the functions and saving the rows in the dataset missing:

missing = get_data(missingids, get_iucn_species)

As before, we should extract the information from the dataframe obtained above by using the function ``extract_spe`` using the relevant colums.

In [186]:
missing.head()

,name,result
0,110847130,"{'taxonid': 110847130, 'scientific_name': 'Mob..."
1,46967827,"{'taxonid': 46967827, 'scientific_name': 'Derm..."
2,161749,"{'taxonid': 161749, 'scientific_name': 'Isurus..."
3,60200,"{'taxonid': 60200, 'scientific_name': 'Mobula ..."
4,41878,"{'taxonid': 41878, 'scientific_name': 'Stegost..."


In [187]:
#Extracting columns

missingc = extract_spe(['taxonid', 'scientific_name', 'published_year', 'assessment_date',  'marine_system', 'freshwater_system', 
                       'terrestrial_system', 'kingdom', 'class', 'category'], missing)

#Dropping original columns
missingc.drop(columns = ["name", "result"], inplace = True)

In [188]:
missingc.head()

,taxonid,scientific_name,published_year,assessment_date,marine_system,freshwater_system,terrestrial_system,kingdom,class,category
0,110847130,Mobula mobular,2019,2018-11-20,True,False,False,ANIMALIA,CHONDRICHTHYES,EN
1,46967827,Dermochelys coriacea Northwest Atlantic Ocean ...,2019,2019-01-18,True,False,True,ANIMALIA,REPTILIA,EN
2,161749,Isurus oxyrinchus Atlantic subpopulation,2009,2004-06-16,True,False,False,ANIMALIA,CHONDRICHTHYES,VU
3,60200,Mobula thurstoni,2019,2018-11-09,True,False,False,ANIMALIA,CHONDRICHTHYES,EN
4,41878,Stegostoma tigrinum,2019,2015-02-18,True,False,False,ANIMALIA,CHONDRICHTHYES,EN


Now we will apply the following function to ``df`` in order to standardize the system for each species. Then, we create dummy variables for each of these systems. 

In [189]:
def get_system(system):
    if system == "Terrestrial":
        return "terrestrial"
    elif system == "Freshwater (=Inland waters)":
        return "freshwater"
    elif system == "Terrestrial|Freshwater (=Inland waters)":
        return "terrestrial, freshwater"
    elif (system == "Marine" or system== "Marine|Marine"):
        return "marine"
    elif system == "Terrestrial|Marine":
        return "terrestrial, marine"
    elif system == "Freshwater (=Inland waters)|Marine":
        return "freshwater, marine"
    elif system == "Terrestrial|Freshwater (=Inland waters)|Marine":
        return "terrestrial, freshwater, marine"
    

In [202]:
df.head()

,assessmentId,taxonid,scientific_name,category,published_year,assessment_date,systems,realm,scopes,kingdom,class
0,497499,132523146,Hubbsina turneri,Critically Endangered,2019,2018-04-17 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
1,500479,11058,Kubaryia pilikia,Critically Endangered,2012,2011-08-22 00:00:00 UTC,Terrestrial,Oceanian,Global,ANIMALIA,GASTROPODA
2,500918,1117,Ameca splendens,Critically Endangered,2019,2018-04-18 00:00:00 UTC,Freshwater (=Inland waters),Neotropical,Global,ANIMALIA,ACTINOPTERYGII
3,502298,11256,Obovaria haddletoni,Critically Endangered,2012,2012-04-11 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA
4,502943,11479,Lemiox rimosus,Critically Endangered,2012,2012-03-12 00:00:00 UTC,Freshwater (=Inland waters),Nearctic,Global,ANIMALIA,BIVALVIA


In [204]:
df["systems"] = df.systems.apply(get_system)

There are some species without system defined. However, after seeing that all of them are plants, we can fill those fields with ``terrestrial``:

In [221]:
df[df.systems.isna()]

,assessmentId,taxonid,scientific_name,category,published_year,assessment_date,systems,realm,scopes,kingdom,class
614,2814360,32318,Abies beshanzuensis,Critically Endangered,2013,2010-12-13 00:00:00 UTC,None,Indomalayan,Global,PLANTAE,PINOPSIDA
621,2829141,32975,Araucaria angustifolia,Critically Endangered,2013,2011-08-16 00:00:00 UTC,None,Neotropical,Global,PLANTAE,PINOPSIDA
3911,43328084,43324365,Phragmipedium exstaminodium,Critically Endangered,2016,2015-09-09 00:00:00 UTC,None,Neotropical,Global,PLANTAE,LILIOPSIDA
3913,43328114,43324837,Phragmipedium kovachii,Critically Endangered,2016,2015-07-30 00:00:00 UTC,None,Neotropical,Global,PLANTAE,LILIOPSIDA
3914,43328134,43325573,Phragmipedium manzurii,Critically Endangered,2016,2015-09-13 00:00:00 UTC,None,Neotropical,Global,PLANTAE,LILIOPSIDA
3916,43328164,43325915,Phragmipedium tetzlaffianum,Critically Endangered,2016,2015-09-18 00:00:00 UTC,None,Neotropical,Global,PLANTAE,LILIOPSIDA
5014,91322572,32758,Varronia urticacea,Critically Endangered,2016,1998-01-03 00:00:00 UTC,None,Neotropical,Global,PLANTAE,MAGNOLIOPSIDA
5015,91322691,30916,Varronia wagnerorum,Critically Endangered,2016,1998-01-03 00:00:00 UTC,None,Neotropical,Global,PLANTAE,MAGNOLIOPSIDA
8783,2810241,32101,Abies fraseri,Endangered,2013,2011-05-11 00:00:00 UTC,None,Nearctic,Global,PLANTAE,PINOPSIDA
8918,2852140,34321,Agathis macrophylla,Endangered,2013,2012-03-28 00:00:00 UTC,None,Australasian|Oceanian,Global,PLANTAE,PINOPSIDA


In [222]:
df.systems.fillna("terrestrial", inplace= True)

The next function will be iterated 3 times in order to get the dummies for the systems:

In [231]:
#Creating dummies encoding the systems. Here we cannot use pd.get_dummies function because each value in systems may contain multiple systems

def get_dum(keyword, word):
    if keyword in word.split(", "):
        return True
    else:
        return False      

#Creating new features
for system in ["marine", "freshwater", "terrestrial"]:
    df[system] = df.systems.apply(lambda x: get_dum(system, x))

# dropping the column systems

df.drop(columns = ["systems"], inplace= True)

In [239]:
df.drop(columns = ["assessmentId", "realm", "scopes"], inplace= True)

In [240]:
df.head()

,taxonid,scientific_name,category,published_year,assessment_date,kingdom,class,marine,freshwater,terrestrial
0,132523146,Hubbsina turneri,Critically Endangered,2019,2018-04-17 00:00:00 UTC,ANIMALIA,ACTINOPTERYGII,False,True,False
1,11058,Kubaryia pilikia,Critically Endangered,2012,2011-08-22 00:00:00 UTC,ANIMALIA,GASTROPODA,False,False,True
2,1117,Ameca splendens,Critically Endangered,2019,2018-04-18 00:00:00 UTC,ANIMALIA,ACTINOPTERYGII,False,True,False
3,11256,Obovaria haddletoni,Critically Endangered,2012,2012-04-11 00:00:00 UTC,ANIMALIA,BIVALVIA,False,True,False
4,11479,Lemiox rimosus,Critically Endangered,2012,2012-03-12 00:00:00 UTC,ANIMALIA,BIVALVIA,False,True,False


Now, finally we can concat our initial dataset ``df`` and ``missing``; after this, we can merge the resultant dataset with ``dfc``:

In [242]:
missing = missing[["taxonid", "scientific_name", "category", "published_year", "assessment_date", "kingdom", "class", "marine_system", "freshwater_system", "terrestrial_system"]]

missing.rename(columns = {"marine_system": "marine", 
                          "freshwater_system": "freshwater", 
                          "terrestrial_system": "terrestrial"}, inplace = True)

,taxonid,scientific_name,category,published_year,assessment_date,kingdom,class,marine,freshwater,terrestrial
0,110847130,Mobula mobular,EN,2019,2018-11-20,ANIMALIA,CHONDRICHTHYES,True,False,False
1,46967827,Dermochelys coriacea Northwest Atlantic Ocean ...,EN,2019,2019-01-18,ANIMALIA,REPTILIA,True,False,True
2,161749,Isurus oxyrinchus Atlantic subpopulation,VU,2009,2004-06-16,ANIMALIA,CHONDRICHTHYES,True,False,False
3,60200,Mobula thurstoni,EN,2019,2018-11-09,ANIMALIA,CHONDRICHTHYES,True,False,False
4,41878,Stegostoma tigrinum,EN,2019,2015-02-18,ANIMALIA,CHONDRICHTHYES,True,False,False


In [244]:
taxondata = pd.concat([df,missing]).reset_index(drop = True)
spe_complete = dfc.merge(taxondata, on = ["taxonid", "scientific_name"], how = "left")

#Dropping and renaming columns
spe_complete.rename(columns = {"category_x": "category"}, inplace= True)
spe_complete.drop(columns = "category_y", inplace = True)


Let us explore ``spe_complete`` by counting the number of NaN values:

In [250]:
null_cols(spe_complete)

Country    0.224511
ISO2       0.224511
ISO3       0.224511
dtype: float64

,taxonid,scientific_name,category,Country,ISO2,ISO3,published_year,assessment_date,kingdom,class,marine,freshwater,terrestrial
0,219,Acinonyx jubatus,VU,Uzbekistan,UZ,UZB,2015,2014-05-28 00:00:00 UTC,ANIMALIA,MAMMALIA,False,False,True
1,251,Acipenser nudiventris Aral Sea stock,EX,Uzbekistan,UZ,UZB,1996,1996-08-01,ANIMALIA,ACTINOPTERYGII,False,True,False
2,162346,Aldrovanda vesiculosa,EN,Uzbekistan,UZ,UZB,2012,2012-04-30 00:00:00 UTC,PLANTAE,MAGNOLIOPSIDA,False,True,False
3,164598,Alsophylax laevis,CR,Uzbekistan,UZ,UZB,2017,2008-12-14 00:00:00 UTC,ANIMALIA,REPTILIA,False,False,True
4,47755582,Alsophylax loricatus,VU,Uzbekistan,UZ,UZB,2017,2016-04-05 00:00:00 UTC,ANIMALIA,REPTILIA,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60448,49845682,Tropidurus xanthochilus,VU,Bolivia (Plurinational State of),BO,BOL,2017,2014-02-25 00:00:00 UTC,ANIMALIA,REPTILIA,False,False,True
60449,44392141,Xyris boliviana,VU,Bolivia (Plurinational State of),BO,BOL,2013,2009-03-18 00:00:00 UTC,PLANTAE,LILIOPSIDA,False,False,True
60450,56433,Yunganastes ashkapara,VU,Bolivia (Plurinational State of),BO,BOL,2004,2004-04-30 00:00:00 UTC,ANIMALIA,AMPHIBIA,False,False,True
60451,56465,Yunganastes bisignatus,EN,Bolivia (Plurinational State of),BO,BOL,2004,2004-04-30 00:00:00 UTC,ANIMALIA,AMPHIBIA,False,False,True


We will center our attention in species from the Kingdom *Animalia* and *Plantae*, so we filter ``spe_complete`` accordingly and do some cosmetic changes:

In [262]:
spe_relevant = spe_complete[spe_complete.kingdom.isin({"ANIMALIA", "PLANTAE"})].copy()

#resetting index

spe_relevant.reset_index(drop = True, inplace = True)

# For better clarity, return to complete words instead abbreviations

status = {"VU": "Vulnerable",
          "EN": "Endangered",
          "CR": "Critically Endangered",
          "EX": "Extinct",
          "EW": "Extinct in the Wild"
          }

spe_relevant["category"] = spe_relevant.category.apply(lambda n: status[n])


In [264]:
spe_relevant.head()

,taxonid,scientific_name,category,Country,ISO2,ISO3,published_year,assessment_date,kingdom,class,marine,freshwater,terrestrial
0,219,Acinonyx jubatus,Vulnerable,Uzbekistan,UZ,UZB,2015,2014-05-28 00:00:00 UTC,ANIMALIA,MAMMALIA,False,False,True
1,251,Acipenser nudiventris Aral Sea stock,Extinct,Uzbekistan,UZ,UZB,1996,1996-08-01,ANIMALIA,ACTINOPTERYGII,False,True,False
2,162346,Aldrovanda vesiculosa,Endangered,Uzbekistan,UZ,UZB,2012,2012-04-30 00:00:00 UTC,PLANTAE,MAGNOLIOPSIDA,False,True,False
3,164598,Alsophylax laevis,Critically Endangered,Uzbekistan,UZ,UZB,2017,2008-12-14 00:00:00 UTC,ANIMALIA,REPTILIA,False,False,True
4,47755582,Alsophylax loricatus,Vulnerable,Uzbekistan,UZ,UZB,2017,2016-04-05 00:00:00 UTC,ANIMALIA,REPTILIA,False,False,True


In order to do a good classification by Biorealms we should include this feature. First we load a dataset containing this information and then we merge it with ``spe_relevant``:

In [265]:
realms = pd.read_csv("../data/raw_data/regions.csv")

In [277]:
spe_realms = (realms[["Realm", "Continent", "Region", "Subregion", "Country", "ISO3", "Developed"]]
              .merge(spe_relevant, on = "ISO3", how = "right")
              .drop(columns = ["Country_y", "ISO2"])
              .rename(columns = {"Country_x": "Country"}))

# Rearranging

spe_realms = spe_realms[['taxonid', 'scientific_name', 'category', 'published_year','assessment_date', 'kingdom', 
                         'class', 'marine', 'freshwater','terrestrial', 'Realm', 'Continent', 'Region', 'Subregion', 
                         'Country', 'ISO3','Developed']]

Exploring the datatypes we realized that ``assessment_date`` doesn't have the data type date, and then we change it and create a new feature containing the year of assessment of the species which differs of the publication year:

In [283]:
spe_realms.dtypes

taxonid             int64
scientific_name    object
category           object
published_year      int64
assessment_date    object
kingdom            object
class              object
marine               bool
freshwater           bool
terrestrial          bool
Realm              object
Continent          object
Region             object
Subregion          object
Country            object
ISO3               object
Developed           int64
dtype: object

In [286]:
spe_realms["assessment_date"] = pd.to_datetime(spe_relevant.assessment_date, infer_datetime_format=True, utc= True)
spe_realms = spe_realms.assign(year = spe_realms["assessment_date"].dt.year)


In [287]:
spe_realms.head()

,taxonid,scientific_name,category,published_year,assessment_date,kingdom,class,marine,freshwater,terrestrial,Realm,Continent,Region,Subregion,Country,ISO3,Developed,year
0,30320,Abies numidica,Critically Endangered,2011,2014-05-28 00:00:00+00:00,PLANTAE,PINOPSIDA,False,False,True,Palearctic,Africa,Northern Africa,Northern Africa,Algeria,DZA,0,2014
1,61455,Acanthodactylus blanci,Endangered,2006,1996-08-01 00:00:00+00:00,ANIMALIA,REPTILIA,False,False,True,Palearctic,Africa,Northern Africa,Northern Africa,Algeria,DZA,0,1996
2,61463,Acanthodactylus spinicauda,Critically Endangered,2006,2012-04-30 00:00:00+00:00,ANIMALIA,REPTILIA,False,False,True,Palearctic,Africa,Northern Africa,Northern Africa,Algeria,DZA,0,2012
3,218,Acilius duvergeri,Vulnerable,1996,2008-12-14 00:00:00+00:00,ANIMALIA,INSECTA,False,True,False,Palearctic,Africa,Northern Africa,Northern Africa,Algeria,DZA,0,2008
4,219,Acinonyx jubatus,Vulnerable,2015,2016-04-05 00:00:00+00:00,ANIMALIA,MAMMALIA,False,False,True,Palearctic,Africa,Northern Africa,Northern Africa,Algeria,DZA,0,2016


In [288]:
#Saving the curated data

spe_realms.to_csv("../data/clean_data/species-realm-curate.csv")

# 2. Temperature data 

In [294]:
temp_anomal = pd.read_csv('../data/raw_data/graph.txt', sep="     ", engine= "python")

In [295]:
temp_anomal.head()

,Year,temp,lowest
0,1880,-0.16,-0.08
1,1881,-0.07,-0.12
2,1882,-0.10,-0.16
3,1883,-0.17,-0.19
4,1884,-0.28,-0.23


In [299]:
prec_temp = pd.read_csv("../data/raw_data/prec_temp.csv")
prec_temp.drop(columns = "Unnamed: 0", inplace= True)

In [301]:
prec_temp.dtypes

Year               int64
ISO3              object
Precipitation    float64
Temperature      float64
Realm             object
Country           object
dtype: object

In [302]:
prec_temp.head()

,Year,ISO3,Precipitation,Temperature,Realm,Country
0,1991,AFG,435.44990,12.894488,Palearctic,Afghanistan
1,1991,ALB,917.86880,11.037649,Palearctic,Albania
2,1991,DZA,88.13276,22.513458,Palearctic,Algeria
3,1991,AND,757.11630,11.188148,Palearctic,Andorra
4,1991,AGO,1017.98660,21.716883,Afrotropical,Angola


In [ ]:
year_realm = (spe_relevant[["year", "Realm", "taxonid"]]
            .groupby(["year", "Realm"])
            .count()
            .reset_index())

In [ ]:
year_realm.head()

In [ ]:
pivot_year_realm = pd.pivot_table(year_realm, values='taxonid', 
                                index=['year'], 
                                columns=['Realm'], 
                                aggfunc=np.sum).reset_index()

#Filling with zeros
pivot_year_realm = pivot_year_realm.fillna(0)

In [ ]:
cols = ['Afrotropical', 'Antarctic', 'Australasian', 'Indomalayan','Nearctic', 'Neotropical', 'Oceanian', 'Palearctic']

for col in cols:
    pivot_year_realm[col] = pivot_year_realm[col].cumsum()

In [ ]:
pivot_year_realm

In [ ]:
# Here we use hvplot library

#Ploting both lines: Variation extent and variation area
"""
j = pivot_cat_year.hvplot.area(x ='yearPublished', 
                       y = ["Extinct in the Wild", "Extinct", 'Critically Endangered', 'Endangered',  "Vulnerable"], 
                       value_label ='Number of species', # counts of species is the numerical feature
                       title = "Animal species in the IUCN red list (2000-2019)",
                       xlabel = "Year",
                       cmap = "Pastel1", # colormap set
                       width =800, 
                       height =400,
                       line_width = 0.5,
                       line_join = "round")

j.opts(legend_position='top_left')
"""

j = (pivot_year_realm.hvplot.line(x= "year", 
                          y= ["Afrotropical", "Antarctic", 'Australasian', 'Indomalayan',  "Nearctic", "Neotropical", "Oceanian", "Palearctic"], 
                          value_label='number of species',
                          title = "Endangered and Extinct species by Bio Realm",
                          xlabel = "year",
                          ylabel = "number of species",
                          #  logy = True, possible to do
                          cmap = "glasbey_cool",
                          width=900, 
                          height=400,
                          line_width = 3,
                          alpha = 0.6))

#positioning legends
j.opts(legend_position='right')

In [ ]:
plants = set(spe_relevant[spe_relevant["kingdom"] == "PLANTAE"]["class"].value_counts().index)


In [ ]:
spe_relevant[spe_relevant["Realm"]== "Australasian"]["class"].value_counts()

In [ ]:
#Plot with HoloViews

temp_an = hv.Curve(global_anomalies, 
                   ('Year', 'Year'), 
                   ('J-D', 'Annual variation'), 
                   label="Temperature Anomalies")

#Line options
temp_an.opts(opts.Curve(height=500, 
                        width=800, 
                        line_width=2, 
                        color="orange", 
                        tools=['hover']))

#baseline plotting
baseline = (hv.HLine(0))

baseline.opts(opts.HLine(color = "cornflowerblue", 
                         line_width = 1, 
                         tools = ["hover"], 
                         line_dash='dashed'))

#Composing the plot
temp_an * baseline * hv.Text(2000, -0.05, 'Baseline temperature 1960')

# Temperatures and Precipitation datasets

Temperature & precipitation datasets:  https://climateknowledgeportal.worldbank.org
We aim to classify or cluster countries in 4 categories:
 * Cold Dry
 * Cold Wet
 * Hot dry
 * Hot wet

Then the idea is to explore the impact of climate change in representatives of those regions including the frequency or vulnerability of suffering natural events.


In [ ]:
temp_anomal.columns = ["Year", "temp", "lowest"]

In [ ]:
temp_anomal.drop([0], inplace= True)

In [ ]:
temp_anomal = temp_anomal.apply(pd.to_numeric)

In [ ]:
temp_anomal.dtypes

In [ ]:
prec_temp = pd.read_csv("data/prec_temp.csv")

In [ ]:
prec_temp.reset_index(drop = True, inplace = True)

In [ ]:
prec_temp.drop(columns = "Unnamed: 0", inplace= True)

In [ ]:
prec_temp.merge()

In [ ]:
years = hv.HoloMap(kdims=['Year'])

for i in range(2000, 2017):
    years[i] = hv.Points(prec_temp[prec_temp.Year == i], 
                   ["Precipitation", "Temperature"],
                   ['Country', 'Year', "Realm", "ISO3"]).sort('Year')
    
    tooltips = [('Country', '@Country'),
            ('Year', '@Year'), ("Realm", "@Realm")
            ]

    hover = HoverTool(tooltips=tooltips)

    years[i].opts(tools=[hover], 
            color='Realm', 
            cmap='Set2',
            line_color='black', 
            padding=0.1, 
            size = 5,
            width=600, 
            height=350, 
            show_grid=True,
            #logx = True,
            title='Temperature vs precipitation')
    
years.opts(legend_position='right')    

In [ ]:
prec_temp.head()

In [ ]:
geom = gpd.read_file('data/countries.geojson')

In [ ]:
geom.head()

In [ ]:
geom.shape

In [ ]:
prec_temp = prec_temp.merge(geom[["ISO_A3", "geometry"]], left_on = "ISO3", right_on = "ISO_A3")

In [ ]:
prec_temp.drop(columns = "ISO_A3", inplace = True)

In [ ]:
prec_temp.dtypes

In [ ]:
from geopandas import GeoDataFrame

gdf = GeoDataFrame(prec_temp, geometry="geometry")

In [ ]:
gdf = gdf[gdf.Year==1991]

In [ ]:
gv.Polygons(gdf, 
            vdims=['Temperature', "Country"]).opts(tools=['hover'],
                                                   width=800,
                                                   height= 500,
                                                   projection=crs.PlateCarree(),
                                                   cmap = "Spectral_r")

In [ ]:
prec_temp.head()

In [ ]:
pd.read_csv("data/temperature.csv")

In [ ]:
pd.read_csv("data/africa.csv")

In [ ]:
emissions_pc = pd.read_csv("data/co-emissions-per-capita.csv")

In [ ]:
emissions_share = pd.read_csv("data/annual-share-of-co2-emissions.csv")

In [ ]:
emissions_share.rename(columns = {"Share of global CO₂ emissions (%)": "share_perc"}, inplace = True)

In [ ]:
emissions_pc.rename(columns = {"Per capita CO₂ emissions (tonnes per capita)": "emissions_pc"}, inplace = True)

In [ ]:
emissions_share.merge(regions[["ISO3"]])

In [ ]:
emissions_share = emissions_share.merge(realms[["Realm", "ISO3"]], left_on = "Code", right_on = "ISO3", how = "left")

In [ ]:
emissions_pc= emissions_pc.merge(realms[["Realm", "ISO3"]], left_on = "Code", right_on = "ISO3", how = "left")

In [ ]:
emissions_share.drop(columns = "ISO3", inplace = True)
emissions_pc.drop(columns = "ISO3", inplace = True)


In [ ]:
emissions_share.head()

In [ ]:
emissions_share.head()

In [ ]:
emissions_share[emissions_share.Code.isna()].Entity.value_counts()

In [ ]:
realms.head()